In [1]:
import numpy as np
import pandas as pd
from fbprophet import Prophet
import itertools


In [2]:
cff = pd.read_csv('~/Downloads/cashflowtemplate.csv')
cff

,year,Inventories,profit,Receivables,ProvisonsCurrent,Payables,OtherCurrentLiabilities,ProfitSaleonIncome,Depreciation,OtherCurrentFinancialLiabilities,...,NonCurrentInvestments,InterestReceived,DividendReceived,OtherNonCurrentAssets,BorrowingsCurrent,BorrowingsNonCurrent,DividendPaid,OtherEquity,InterestPaid,TotalTaxExpense
0,01/04/16,26380.0,22267.0,684.0,37.0,760.0,8334.0,54.0,1049.0,235.0,...,13500,67.0,8.0,3000,1049.0,55.0,1500,7500,1550,4000
1,01/07/16,26400.0,22250.0,700.0,35.0,750.0,8330.0,54.0,1040.0,235.0,...,13600,61.0,9.0,3200,1040.0,57.0,1510,7650,1600,4200
2,01/10/16,26370.0,22300.0,680.0,36.0,760.0,8331.0,54.0,1050.0,235.0,...,13700,-25.0,8.0,3250,1050.0,50.0,155,9000,1000,4400
3,01/01/17,26380.0,22267.0,684.0,37.0,760.0,8333.0,54.0,1049.0,235.0,...,13800,-24.0,9.0,3300,1049.0,55.0,1560,6500,500,4600
4,01/04/17,27861.0,24355.0,1988.0,9.0,1276.0,5004.0,64.0,990.0,3180.0,...,15000,-969.0,1700.0,3300,2000.0,66.0,1530,2000,350,5000
5,01/07/17,27870.0,24340.0,1980.0,9.0,1270.0,5000.0,64.0,995.0,3190.0,...,15200,-960.0,105.0,2350,2200.0,60.0,1520,500,300,5300
6,01/10/17,27867.0,24345.0,2000.0,10.0,1280.0,5001.0,64.0,990.0,3200.0,...,15400,-1000.0,1706.0,3400,2400.0,67.0,1550,1000,500,5600
7,01/01/18,27861.0,24355.0,1988.0,10.0,1276.0,5004.0,64.0,990.0,3180.0,...,15600,-940.0,1707.0,3450,2600.0,66.0,1560,500,1000,5900
8,01/04/18,27506.0,25716.0,592.0,19.0,1211.0,1321.0,363.0,788.0,3486.0,...,19000,332.0,1720.0,3450,3600.0,15000.0,1570,2000,1500,6000
9,01/07/18,27500.0,25720.0,590.0,20.0,1210.0,1320.0,363.0,790.0,3490.0,...,19300,370.0,1721.0,3500,3900.0,15300.0,1560,22000,1600,6200


In [3]:
idx = pd.date_range('03/31/2016', periods = 12, freq = '3M')
idx

DatetimeIndex(['2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',
               '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',
               '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31'],
              dtype='datetime64[ns]', freq='3M')

In [4]:
cff['Year'] = idx

In [5]:
def change(data):
    x = pd.DataFrame()
    x['year'] = data[data.columns[0]]
    x[data.columns[1]] = data[data.columns[1]].diff(1).fillna(0)
    return x

In [6]:
def forecast1(data):
    e2 = pd.DataFrame()
    e2['ds'] = data[data.columns[0]]
    e2['y'] = data[data.columns[1]]
    m = Prophet()
    m.fit(e2)
    future = m.make_future_dataframe(periods = 2, freq='3M')
    forecast = m.predict(future)
    f = forecast['yhat'].tail(2)
    return f

In [7]:
def ins_o(o_ins_in1_m, o_ins_out1_m):
    q1in1 = []
    q1out1 = []
    q2in1 = []
    q2out1 = []
    
            
    for f in o_ins_in1_m[::2]:
        q1in1.append(f)
        
    for v in o_ins_in1_m[1::2]:
        q2in1.append(v)
            
    for f in o_ins_out1_m[::2]:
        q1out1.append(f)
        
    for v in o_ins_out1_m[1::2]:
        q2out1.append(v)
        
   
    q1in = sum(q1in1)
    q2in = sum(q2in1)
    q1out = sum(q1out1)
    q2out = sum(q2out1)
    
    
    ins1 = q1in - q1out
    ins2 = q2in - q2out
        
    return ins1, ins2

In [8]:
def ins(ins1_m):
    q1ins1 = []
    q2ins1 = []
    
            
    for f in ins1_m[::2]:
        q1ins1.append(f)
        
    for v in ins1_m[1::2]:
        q2ins1.append(v)
            
   
    q1ins = sum(q1ins1)
    q2ins = sum(q2ins1)
    
    return q1ins, q2ins

In [9]:
def in_out_o(asset, liability):
    
    q1in = []
    q1out = []
    q2in = []
    q2out = []
    q1in_f = []
    q1out_f = []
    q2in_f = []
    q2out_f = []
            
            
    for f in asset[::2]:
        q1in.append(f)
        
    for v in asset[1::2]:
        q2in.append(v)
            
    for f in liability[::2]:
        q1out.append(f)
        
    for v in liability[1::2]:
        q2out.append(v)
        
    for i in q1in:
        if i > 0:
            q1in_f.append(i)
        else:
            q1out_f.append(i)
            
    for j in q1out:
        if j <= 0:
            q1in_f.append(j)
        else:
            q1out_f.append(j)
            
    for i in q2in:
        if i > 0:
            q2in_f.append(i)
        else:
            q2out_f.append(i)
            
    for j in q2out:
        if j <= 0:
            q2in_f.append(j)
        else:
            q2out_f.append(j)
        
    q1in_a = [abs(x) for x in q1in_f]
    q2in_a = [abs(x) for x in q2in_f]
    q1out_a = [abs(x) for x in q1out_f]
    q2out_a = [abs(x) for x in q2out_f]
    
    q1in_s = sum(q1in_a)
    q2in_s = sum(q2in_a)
    q1out_s = sum(q1out_a)
    q2out_s = sum(q2out_a)
    
    
    bs1 = q1in_s - q1out_s
    bs2 = q2in_s - q2out_s
        
    return bs1, bs2

In [10]:
def in_out_i(asset):
    
    q1ass = []
    q2ass = []
    q1in_f = []
    q1out_f = []
    q2in_f = []
    q2out_f = []
            
            
    for f in asset[::2]:
        q1ass.append(f)
        
    for v in asset[1::2]:
        q2ass.append(v)
    
    for i in q1ass:
        if i < 0:
            q1in_f.append(i)
        else:
            q1out_f.append(i)
            
    for i in q2ass:
        if i < 0:
            q2in_f.append(i)
        else:
            q2out_f.append(i)
            
        
    q1in_a = [abs(x) for x in q1in_f]
    q2in_a = [abs(x) for x in q2in_f]
    q1out_a = [abs(x) for x in q1out_f]
    q2out_a = [abs(x) for x in q2out_f]
    
    q1in_s = sum(q1in_a)
    q2in_s = sum(q2in_a)
    q1out_s = sum(q1out_a)
    q2out_s = sum(q2out_a)
    
    
    bs1 = q1in_s - q1out_s
    bs2 = q2in_s - q2out_s
        
    return bs1, bs2

In [11]:
def in_out_f(liability):
    
    q1lib = []
    q2lib = []
    q1in_f = []
    q1out_f = []
    q2in_f = []
    q2out_f = []
            
            
    for f in liability[::2]:
        q1lib.append(f)
        
    for v in liability[1::2]:
        q2lib.append(v)
    
    for i in q1lib:
        if i > 0:
            q1in_f.append(i)
        else:
            q1out_f.append(i)
            
    for i in q2lib:
        if i < 0:
            q2in_f.append(i)
        else:
            q2out_f.append(i)
            
        
    q1in_a = [abs(x) for x in q1in_f]
    q2in_a = [abs(x) for x in q2in_f]
    q1out_a = [abs(x) for x in q1out_f]
    q2out_a = [abs(x) for x in q2out_f]
    
    q1in_s = sum(q1in_a)
    q2in_s = sum(q2in_a)
    q1out_s = sum(q1out_a)
    q2out_s = sum(q2out_a)
    
    
    bs1 = q1in_s - q1out_s
    bs2 = q2in_s - q2out_s
        
    return bs1, bs2

In [12]:
def ocf (profit1, profit2, o1, o2, oi1, oi2, tax1, tax2):
    a1 = o1 + oi1 + tax1
    a2 = o2 + oi2 + tax2
    
    a1_a = abs(a1) 
    a2_a = abs(a2)
    
    oq1 = profit1 - a1_a
    oq2 = profit2 - a2_a
    
    return oq1, oq2

In [13]:
def icf(i1, i2, ii1, ii2):
    iq1 = i1 + ii1
    iq2 = i2 + ii2

    return iq1, iq2

In [14]:
def fcf(f1, f2, fi1, fi2):
    fq1 = f1 - fi1
    fq2 = f2 - fi2

    return fq1, fq2

In [15]:
def nc(oq1, oq2, iq1, iq2, fq1, fq2):
    ncq1 = oq1 + iq1 + fq1
    ncq2 = oq2 + iq2 + fq2
    
    return ncq1, ncq2

# Operating Cash Flow

In [16]:
inv = cff[['Year', 'Inventories']]
rec = cff[['Year', 'Receivables']]
pay = cff[['Year', 'Payables']]
ocl = cff[['Year', 'OtherCurrentLiabilities']]
ii = cff[['Year', 'InterestIncome']]
ie = cff[['Year', 'InterestExpense']] 
di = cff[['Year', 'DividendIncome']]
psi = cff[['Year', 'ProfitSaleonIncome']] 
dep = cff[['Year', 'Depreciation']]
ofl = cff[['Year', 'OtherCurrentFinancialLiabilities']]
ofa = cff[['Year', 'OtherCurrentFinancialAssets']]
pro = cff[['Year', 'ProvisonsCurrent']]
profit = cff[['Year', 'profit']]
tax = cff[['Year', 'TotalTaxExpense']]

o_ass = [inv, rec, ofa]
o_lib = [pay, ocl, ofl, pro]
o_ins_in = [dep, ie]
o_ins_out = [ii, di, psi]


In [17]:
o_ass_chage = []
o_lib_chage = []

for i in o_ass:
    o_ass_chage.append(change(i))
    
for i in o_lib:
    o_lib_chage.append(change(i))
    

In [18]:
o_ass_forec = []
o_lib_forec = []
o_ins_in_forec = []
o_ins_out_forec = []

for i in o_ass_chage:
    o_ass_forec.append((forecast1(i),i.columns[1]))

for i in o_lib_chage:
    o_lib_forec.append((forecast1(i),i.columns[1]))

for i in o_ins_in:
    o_ins_in_forec.append((forecast1(i),i.columns[1]))
    
for i in o_ins_out:
    o_ins_out_forec.append((forecast1(i),i.columns[1])) 
    
profit_forec = forecast1(profit)
tax_forec = forecast1(tax)
    
profit_forec 

/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/fbprophet/forecaster.py:794: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 8.0.
/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/fbprophet/forecaster.py:353: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  np.linspace(0, hist_size - 1, self.n_changepoints + 1)
/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.flo

12    26621.637849
13    26792.257650
Name: yhat, dtype: float64

In [19]:
profit_l = profit_forec.to_list()
tax_1 = tax_forec.tolist()

inv1 = o_ass_forec[0][0].to_list()
rec1 = o_ass_forec[1][0].to_list()
ofa1 = o_ass_forec[2][0].to_list()

pay1 = o_lib_forec[0][0].to_list()
ocl1 = o_lib_forec[1][0].to_list()
ofl1 = o_lib_forec[2][0].to_list()
pro1 = o_lib_forec[3][0].to_list()

dep1 = o_ins_in_forec[0][0].to_list()
ie1 = o_ins_in_forec[1][0].to_list()

ii1 = o_ins_out_forec[0][0].to_list()
di1 = o_ins_out_forec[1][0].to_list()
psi1 = o_ins_out_forec[0][0].to_list()


o_ass1 = [inv1, rec1, ofa1]
o_ass1_m = list(itertools.chain.from_iterable(o_ass1))

o_lib1 = [pay1, ocl1, ofl1, pro1]
o_lib1_m = list(itertools.chain.from_iterable(o_lib1))

o_ins_in1 = [dep1, ie1]
o_ins_in1_m = list(itertools.chain.from_iterable(o_ins_in1))

o_ins_out1 = [ii1, di1, psi1]
o_ins_out1_m = list(itertools.chain.from_iterable(o_ins_out1))


In [20]:
oi1, oi2 = ins_o(o_ins_in1_m, o_ins_out1_m)

print(oi1, oi2)

-3086.6346236248637 -2496.8333185122656


In [21]:
o1, o2 = in_out_o(o_ass1_m, o_lib1_m)

print(o1, o2)

1477.936169850188 239.11451736296743


In [22]:
oq1, oq2 = ocf(profit_l[0], profit_l[1],o1, o2, oi1, oi2, tax_1[0], tax_1[1])

print(oq1, oq2)

21726.722765069673 22287.87809670325


# Investing Cah Flow

In [23]:
ppe = cff[['Year', 'ppe']]
lon = cff[['Year', 'Loans']]
oca = cff[['Year', 'OtherCurrentAssets']]
ocna = cff[['Year', 'OtherNonCurrentAssets']]
ip = cff[['Year', 'InvestmentProperty']]
nci = cff[['Year', 'NonCurrentInvestments']]
dr = cff[['Year', 'DividendReceived']]
ir = cff[['Year', 'InterestReceived']]

i_ins = [dr, ir]
i_ass = [ppe, lon, oca, ocna, ip, nci]

In [24]:
i_ass_chage = []

for i in i_ass:
    i_ass_chage.append(change(i))

In [25]:
i_ass_forec = []
i_ins_forec = []

for i in i_ass_chage:
    i_ass_forec.append((forecast1(i),i.columns[1]))

for i in i_ins:
    i_ins_forec.append((forecast1(i),i.columns[1]))
    

/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/fbprophet/forecaster.py:794: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 8.0.
/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/fbprophet/forecaster.py:353: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  np.linspace(0, hist_size - 1, self.n_changepoints + 1)
/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.flo

In [26]:
ppe1 = i_ass_forec[0][0].to_list()
lon1 = i_ass_forec[1][0].to_list()
oca1 = i_ass_forec[2][0].to_list()
ocna1 = i_ass_forec[3][0].to_list()
ip1 = i_ass_forec[4][0].to_list()
nci1 = i_ass_forec[5][0].to_list()

dr1 = i_ins_forec[0][0].to_list()
ir1 = i_ins_forec[1][0].to_list()

i_ass1 = [ppe1, lon1, oca1, ocna1, ip1, nci1]
i_ass1_m = list(itertools.chain.from_iterable(i_ass1))

i_ins1 = [dr1, ir1]
i_ins1_m = list(itertools.chain.from_iterable(i_ins1))


In [27]:
i1,i2 = in_out_i(i_ass1_m)

print(i1,i2)

-22735.194066205728 -4295.19841187334


In [28]:
ii1,ii2 = ins(i_ins1_m)

print(ii1,ii2)

2369.9430523350543 1793.9993381527463


In [29]:
iq1, iq2 = icf(i1, i2, ii1, ii2)
print(iq1, iq2)

-20365.251013870675 -2501.1990737205933


# Financing Cash Flow

In [30]:
oe = cff[['Year', 'OtherEquity']]
bc = cff[['Year', 'BorrowingsCurrent']]
bnc = cff[['Year', 'BorrowingsNonCurrent']]
dp = cff[['Year', 'DividendPaid']]
ip = cff[['Year', 'InterestPaid']]

f_ins = [dp, ip]
f_lib = [oe, bc, bnc]

In [31]:
f_lib_chage = []

for i in f_lib:
    f_lib_chage.append(change(i))

In [32]:
f_lib_forec = []
f_ins_forec = []

for i in f_lib_chage:
    f_lib_forec.append((forecast1(i),i.columns[1]))

for i in f_ins:
    f_ins_forec.append((forecast1(i),i.columns[1]))


/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/fbprophet/forecaster.py:794: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 8.0.
/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/fbprophet/forecaster.py:353: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  np.linspace(0, hist_size - 1, self.n_changepoints + 1)
/Users/ramakrishnareddych/anaconda3/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.flo

In [33]:
oe1 = f_lib_forec[0][0].to_list()
bc1 = f_lib_forec[1][0].to_list()
bnc1 = f_lib_forec[2][0].to_list()

dp1 = f_ins_forec[0][0].to_list()
ip1 = f_ins_forec[1][0].to_list()

f_lib1 = [oe1, bc1, bnc1]
f_lib1_m = list(itertools.chain.from_iterable(f_lib1))

f_ins1 = [dp1, ip1]
f_ins1_m = list(itertools.chain.from_iterable(f_ins1))


In [34]:
f1, f2 = in_out_f(f_lib1_m)

print(f1, f2)

8057.989082411553 -9738.754524387321


In [35]:
fi1, fi2 = ins(f_ins1_m)

print(fi1, fi2)

3065.0337512099086 3101.8911005955306


In [36]:
fq1, fq2 = fcf(f1, f2, fi1, fi2)

print(fq1, fq2)

4992.955331201644 -12840.645624982852


# Net Cash

In [37]:
ncq1, ncq2 = nc(oq1, oq2, iq1, iq2, fq1, fq2)

print(ncq1, ncq2)

6354.427082400643 6946.033397999807
